<a href="https://colab.research.google.com/github/KushangShah/Image-Classification-using-ML/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("salader/dogs-vs-cats")

print("Path to dataset files:", path)

100%|██████████| 1.06G/1.06G [00:36<00:00, 31.1MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/salader/dogs-vs-cats/versions/1


In [2]:
!ls -lah /root/.cache/kagglehub/datasets/salader/dogs-vs-cats/versions/1
!mkdir -p ~/datasets
!mv /root/.cache/kagglehub/datasets/salader/dogs-vs-cats/versions/1 ~/datasets/dogs-vs-cats
!ls -lah ~/datasets/dogs-vs-cats

total 20K
drwxr-xr-x 5 root root 4.0K Mar 14 09:52 .
drwxr-xr-x 3 root root 4.0K Mar 14 09:51 ..
drwxr-xr-x 4 root root 4.0K Mar 14 09:51 dogs_vs_cats
drwxr-xr-x 4 root root 4.0K Mar 14 09:52 test
drwxr-xr-x 4 root root 4.0K Mar 14 09:52 train
total 20K
drwxr-xr-x 5 root root 4.0K Mar 14 09:52 .
drwxr-xr-x 3 root root 4.0K Mar 14 09:52 ..
drwxr-xr-x 4 root root 4.0K Mar 14 09:51 dogs_vs_cats
drwxr-xr-x 4 root root 4.0K Mar 14 09:52 test
drwxr-xr-x 4 root root 4.0K Mar 14 09:52 train


In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# For CNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [4]:
# Define dataset paths
train_dir = "/root/datasets/dogs-vs-cats/train"
test_dir = "/root/datasets/dogs-vs-cats/test"

In [5]:
# Data Augmentation for better generalization
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

# Load training and validation sets
train_data = datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset="training"
)

val_data = datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset="validation"
)

Found 16000 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


## CNN Model

In [9]:
# Create the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [11]:
# Model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 34, 34, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 36992)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      18,940,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,034,177 (72.61 MB)

 Trainable params: 19,034,177 (72.61 MB)

 Non-trainable params: 0 (0.00 B)

## Train The Model

In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1009s 2s/step - accuracy: 0.5483 - loss: 0.7105 - val_accuracy: 0.6345 - val_loss: 0.6391
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1000s 2s/step - accuracy: 0.6476 - loss: 0.6360 - val_accuracy: 0.6877 - val_loss: 0.5916
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 981s 2s/step - accuracy: 0.6991 - loss: 0.5765 - val_accuracy: 0.7262 - val_loss: 0.5407
Epoch 4/10
 20/500 ━━━━━━━━━━━━━━━━━━━━ 14:35 2s/step - accuracy: 0.6977 - loss: 0.5767